# [Advent of Code 2020](https://adventofcode.com/2020)

# The toolbox


Generalised pieces of code that either can be used in multiple questions or that simply makes understand the implementation easier.

In [759]:
import math
import operator
import re
from collections import Counter, defaultdict, deque, namedtuple
from functools import partial, reduce
from itertools import chain, cycle, count, islice, starmap, product
from io import StringIO


def Input(day, parser=str.strip, whole_file=False):
    "Fetch the data input from disk."
    filename = f'../data/advent2020/input{day}.txt'
    with open(filename) as fin:
        return mapt(parser, fin)

    
def mapt(fn, *args): 
    "Do a map, and convert the results to a tuple"
    return tuple(map(fn, *args))


def quantify(data, predictate=bool):
    "Count how many items in an iterable have predicated(item) True"
    return sum(map(predictate, data))


def nth(iterable, n, default=None):
    "Returns the nth item or a default value"
    return next(islice(iterable, n, None), default)


cat = lambda s: ''.join(s)

# coordindate deltas for 8 neighbours
NEIGHBOUR8_DELTAS = (
    (-1, -1), (-1, 0), (-1, 1),
    ( 0, -1),          ( 0, 1),
    ( 1, -1), ( 1, 0), ( 1, 1),
)


def neighbours8(x, y):
    ""
    return tuple(
        (x + dx, y + dy)
        for dx, dy in NEIGHBOUR8_DELTAS
    )

## [Day 1: Report Repair](https://adventofcode.com/2020/day/1)

As usual, things start off easy enough, nested looping with a computationally simple upper bound. Finding numbers that add to 2020 and then returning their multiple.

In [441]:
def find_2020(nums: [int]) -> int:
    for index, n1 in enumerate(nums):
        for n2 in nums[index:]:
            if n1 + n2 == 2020:
                return n1 * n2
    raise Exception('Unable to find solution')

In [442]:
data1 = Input(1, int)
find_2020(data1)

1006176

In [443]:
assert _ == 1006176, 'Day 1.1'

In [444]:
# part 2
def find_2020_3(nums: [int]) -> int:
    for index, n1 in enumerate(nums):
        for jindex, n2 in enumerate(nums[index+1:]):
            for n3 in nums[jindex+1:]:
                if n1 + n2 + n3 == 2020:
                    return n1 * n2 * n3
    raise Exception('Unable to find solution')

find_2020_3(data1)

199132160

In [445]:
assert _ == 199132160, 'Day 1.2'

# [Day 2: Password Philosophy](https://adventofcode.com/2020/day/2)

In [446]:
def is_valid_password(password: str, rule: (str, int, int)) -> bool:
    match, lower, upper = rule
    occurance = quantify(password, lambda c: c == match)
    return lower <= occurance <= upper

In [447]:
def password_and_match(line):
    chunks = re.sub('[-:]', ' ', line).strip().split(' ')
    lower, upper, match, _, password = chunks
    return password, (match, int(lower), int(upper))

data2 = Input(2, password_and_match)
quantify(starmap(is_valid_password, data2))

600

In [448]:
assert _ == 600, 'Day 2.1'

In [449]:
# part 2
def is_valid_password2(password: str, rule: (str, int, int)) -> bool:
    match, lower, upper = rule
    is_match = lambda n: password[n - 1] == match
    matches = int(is_match(lower)) + int(is_match(upper))
    return matches == 1

quantify(starmap(is_valid_password2, data2))

245

In [450]:
assert _ == 245, 'Day 2.2'

# [Day 3: Toboggan Trajectory](https://adventofcode.com/2020/day/3)

Here we can simply make use of some modulo arithmitic to loop back through the row when we attempt to index it out of bounds.

In [451]:
def count_trees(grid: [str], x_step: int, y_step: int):
    path = []
    for index, row in enumerate(grid[::y_step]):
        char_index = (x_step * index) % len(row)
        path.append(row[char_index])
    return quantify(path, lambda c: c == '#')
    

data3 = Input(3)
count_trees(data3, 3, 1)

151

In [452]:
assert _ == 151, 'Day 4.1'

In [453]:
# part 2
count_trees_3 = partial(count_trees, data3)
steps = ((1, 1), (3, 1), (5, 1), (7, 1), (1, 2))
reduce(
    operator.mul,
    starmap(count_trees_3, steps)
)

7540141059

In [454]:
assert _ == 7540141059, 'Day 3.2'

# [Day 4: Passport Processing](https://adventofcode.com/2020/day/4)

This one was a bit fiddly in terms of the regexes, I had an error thanks to not ensuring the match ended at the end of a line (i.e. missing a `$`.)

In [455]:
validators = {
    'byr': lambda x: 1920 <= int(re.match('\d{4}$', x)[0]) <= 2002,
    'iyr': lambda x: 2010 <= int(re.match('\d{4}$', x)[0]) <= 2020,
    'eyr': lambda x: 2020 <= int(re.match('\d{4}$', x)[0]) <= 2030,
    'hgt': lambda x: (
        150 <= int(re.match('(\d+)(?:cm|in)$', x)[1]) <= 193
        if re.match('\d+(cm|in)$', x)[1] == 'cm' else
        59 <= int(re.match('(\d+)(?:cm|in)$', x)[1]) <= 76
    ),
    'hcl': lambda x: bool(re.match('#[0-9a-f]{6}$', x)),
    'ecl': lambda x: bool(re.match('amb|blu|brn|gry|grn|hzl|oth$', x)),
    'pid': lambda x: bool(re.match('\d{9}$', x)),
    'cid': lambda x: True
}

def is_valid_passport(passport, validate_fields=False):
    required = set(['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid'])
    if len(required - set([key for key, _ in passport])) > 0:
        return False

    if validate_fields:
        try:
            return all(
                validators[key](value)
                for key, value
                in passport
            )
        except TypeError:
            # regexes may not match, and will error if we attempt to index them
            return False
    return True


def parse_data(data):
    passports = [[]]
    for line in data:
        if not line.strip():
            # Move onto the next passport
            passports.append([])
            continue
        passports[-1].extend(
            re.findall(r'(\w+):([\w#\d]+)\b', line)
        )
        
    if not passports[-1]:
        passports.pop()
    
    return passports

In [456]:
input4 = parse_data(Input(4))
quantify(map(is_valid_passport, input4))

235

In [457]:
assert _ == 235, 'Day 4.1'

In [458]:
# part 2
is_valid_passport_strict = lambda passport: is_valid_passport(passport, True)
quantify(map(is_valid_passport_strict, input4))

194

In [459]:
assert _ == 194, 'Day 4.2'

# [Day 5: Binary Boarding](https://adventofcode.com/2020/day/5)

Today's one invovled writing a reverse binary search, rather than looking for a value in a sorted list, we're instead given the steps to perform to find our seat.

For the second part we just loop through the list of sorted seat IDs until we find a gap, the gap being our seat.


In [460]:
def reverse_binary(steps: str, lower: int, upper: int) -> int:
    step = steps[0]
    pivot = lower + (upper - lower) // 2
    if step in ('F', 'L'):
        if len(steps) == 1:
            return lower
        return reverse_binary(steps[1:], lower, pivot)
    else:
        if len(steps) == 1:
            return upper
        return reverse_binary(steps[1:], pivot + 1, upper)

def docode_boarding_pass(boarding_pass: str) -> (int, int, int):
    row = reverse_binary(boarding_pass[:7], 0, 127)
    col = reverse_binary(boarding_pass[7:], 0, 7)
    seat_id = row * 8 + col
    return (row, col, seat_id)

In [461]:
input5 = Input(5)

decoded_passes = mapt(docode_boarding_pass, input5)
seat_ids = tuple(seat_id for _, _, seat_id in decoded_passes)
max(seat_ids)

926

In [462]:
assert _ == 926, 'Day 5.1'

In [463]:
# part 2
def find_missing_seat_id(seat_ids):
    sorted_ids = sorted(seat_ids)
    prev = sorted_ids[0]

    for seat_id in sorted_ids[1:]:
        if seat_id - 2 == prev:
            return seat_id - 1
        
        prev = seat_id

find_missing_seat_id(seat_ids)

657

In [464]:
assert _ == 657, 'Day 5.2'

# [Day 6: Custom Customs](https://adventofcode.com/2020/day/6)

Today's question was pretty straightforward, just involving some set-wise combinations. The answers I've given are quite terse (and in fact, slower than they're for-loop counterparts), but I enjoy them!

In [465]:
def group_answers(lines):
    groups = [[]]
    for line in lines:
        line = line.strip()
        if not line:
            groups.append([])
            continue
        groups[-1].append(line)
    
    if not groups[-1]:
        groups.pop()
    
    return groups

input6 = group_answers(Input(6))

In [500]:
sum(
    map(
        lambda d: len(reduce(operator.or_, map(set, d))),
        input6
    )
)

6662

In [467]:
assert _ == 6662, 'Day 6.1'

In [498]:
# part 2
sum(
    map(
        lambda d: len(reduce(operator.and_, map(set, d))),
        input6
    )
)

CPU times: user 2.06 ms, sys: 0 ns, total: 2.06 ms
Wall time: 2.07 ms


3382

In [469]:
assert _ == 3382, 'Day 6.2'

# [Day 7: Handy Haversacks](https://adventofcode.com/2020/day/7)

This is the first tree-like problem of the year! I've implemented an [adjacency list](https://en.wikipedia.org/wiki/Adjacency_list) to keep track of the relations, but this means having to invert the list for the first part. Perhaps a more generic tree would have been useful, we'll see if it comes up again!

In [615]:
def parse_rules(rules):
    contents = defaultdict(list)
    color_search = re.compile('(\d+) (\w+\s\w+) bags?,?\.?')
    for rule in rules:
        bag_colour = cat(rule.split(' ')[:2])
        for num, colour in color_search.findall(rule):
            contents[bag_colour].append((int(num), colour))
    return contents



def count_allowed_bags(initial_colour, rules):
    inverted_rules = defaultdict(list)

    for container_colour, allowed_bags in rules.items():
        for _, bag_colour in allowed_bags:
            inverted_rules[bag_colour].append(container_colour)

    allowed = set()
    to_follow = inverted_rules[initial_colour][:]
    while to_follow:
        colour = to_follow.pop(0)
        allowed.add(colour)
        to_follow.extend(inverted_rules[colour])

    return len(allowed)

In [628]:
input7 = parse_rules(Input(7))
count_allowed_bags('shiny gold', input7)

370

In [584]:
assert _ == 370, 'Day 7.1'

In [595]:
# part 2
def count_required_bags(initial_colour, rules):
    to_follow = [(1, initial_colour)]
    required = 0

    while to_follow:
        num, colour = to_follow.pop(0)
        
        for next_rule in rules[colour]:
            # rather than duplicate the rule num times, just duplicate
            # the number of bags
            next_num, next_bag_colour = next_rule
            to_follow.append((next_num * num, next_bag_colour))
        required += num

    # the amount will be off by 1 as we count the initial bag
    return required - 1

In [629]:
count_required_bags('shiny gold', input7)

29547

In [605]:
assert _ == 29547, 'Day 7.2'

# [Day 8: Handheld Halting](https://adventofcode.com/2020/day/8)

This is the first code parsing puzzle of the year, if previous years are anything to go by this may not be the last! Nothing particular complex here, just parse the operations and perform them, exiting when a loop is detected or we've reached the end of the program.

In [678]:
def parse_op(line: str) -> (str, int):
    op, num = re.match('(\w+) \+?(-?\d+)', line).groups()
    return (op, int(num))


def run_ops(ops) -> (int, str):
    accumulator = 0
    op_index = 0
    
    seen = set()
    exit_code = None

    while True:
        if op_index in seen:
            exit_code = 'loop'
            break

        if op_index == len(ops):
            exit_code = 'exit'
            break

        step = 1
        op, val = ops[op_index]

        if op == 'acc':
            accumulator += val
        elif op == 'jmp':
            step = val
        
        seen.add(op_index)
        op_index += step

    return accumulator, exit_code


inputs8 = Input(8, parse_op)
run_ops(inputs8)[0]

1200

In [656]:
assert _ == 1200, 'Day 8.1'

In [681]:
def modify_instructions(ops: (int,)) -> int:
    for index, (op, val) in enumerate(ops):
        if op == 'jmp':
            new_op = 'noop'
        elif op == 'noop':
            new_op = 'jmp'
        else:
            continue

        new_ops = list(ops)
        new_ops[index] = (new_op, val)
    
        accumulator, exit_code = run_ops(new_ops)

        if exit_code == 'exit':
            return accumulator
    raise Exception('Unable to find solution')

modify_instructions(inputs8)

1023

In [654]:
assert _ == 1023, 'Day 8.2'

# [Day 9: Encoding Error](https://adventofcode.com/2020/day/9)

Cracking the XMAS cipher, fun! For the second part of this problem I precalculate the sums that will be required by adding each value in the sequnce to all the values that appeared before it. This means we can find the sum of any range of values by subtracting the lower value in the range from the top.

Granted, this would stil execute quite quickly given the size of the dataset, but it's still a nicer solution.

In [719]:
def is_valid_xmas_data(index, sequence, preamble_length=25):
    if index < preamble_length:
        raise Exception('Index must be larger than the preamble')

    target = sequence[index]
    start = index - preamble_length

    for i, n1 in enumerate(sequence[start:index], start):
        for n2 in sequence[i:index]:
            if n1 + n2 == target:
                return True
    return False


def find_first_non_matching_xmas(sequence, preamble_length=25):
    for index in range(preamble_length, len(sequence)):
        if not is_valid_xmas_data(index, sequence, preamble_length):
            return sequence[index]
    raise Exception('Solution not found')

In [718]:
input9 = Input(9, int)
find_first_non_matching_xmas(input9)

1309761972

In [ ]:
assert _ == 1309761972, 'Day 9.1'

In [720]:
# part 2
def find_xmas_encryption_weakness(target, sequence):
    # precompute all the sums we'll require from the list[]
    sums = [sequence[0]]
    for n in sequence:
        sums.append(n + sums[-1])
    
    for start_index, sum_start in enumerate(sums):
        for end_index, sum_end in enumerate(sums[start_index + 1:], start_index):
            if sum_end - sum_start == target:
                sequence_slice = sequence[start_index:end_index + 1]
                return min(sequence_slice) + max(sequence_slice)

            if sum_end - sum_start > target:
                break
    raise Exception('Solution not found')

find_xmas_encryption_weakness(1309761972, input9)

177989832

In [711]:
assert _ == 177989832, 'Day 9.2'

# [Day 10: Adapter Array](https://adventofcode.com/2020/day/10)

I solved this problem using [dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming) which is a fancy way of saying that pure-function calls are memoized and therefore not repeatedly called. In this case, rather than looking forward when counting paths ("how many ways can I arrive at X?") we look back ("How many ways can I get here?"). This allows us to make use of previous results and simply combine them at each new step.

In [736]:
def find_joltage_chain_diffs(adapters):
    adapters = sorted(adapters)
    # add the built in power adapters
    adapters.append(adapters[-1] + 3)

    joltage = 0

    diffs = {1: 0, 3: 0}

    for adapter in adapters:
        diff = adapter - joltage
        diffs[diff] += 1

        joltage = adapter
    
    return diffs

def multiply_diffs(adapters):
    diffs = find_joltage_chain_diffs(adapters)
    return diffs[1] * diffs[3]

In [737]:
adapters = Input(10, int)
multiply_diffs(adapters)

2112

In [738]:
assert _ == 2112, 'Day 10.1'

In [149]:
# part 2
def count_combinations(adapters):
    adapters = sorted(adapters)
    adapters.append(adapters[-1] + 3)
    adapters.insert(0, 0)

    chains = {0: 1}

    # For each adadpter, find the number of paths for the numbers
    # _before_ it and combine
    for index, adapter in enumerate(adapters):
        start_index = max(index - 4, 0)
        for prev_adapter in adapters[start_index:index]:
            if adapter - prev_adapter > 3:
                # Cannot be reached, so skip
                continue
            chains[adapter] = chains.get(adapter, 0) + chains[prev_adapter]
    
    return chains[adapters[-1]]

count_combinations(adapters)

3022415986688

In [824]:
assert _ == 3022415986688, 'Day 10.2'

# [Day 11: Seating System](https://adventofcode.com/2020/day/11)

Cellular automata! Always fun to explore. Common pitfalls with this type of problem is updating the same grid, not cloning a new one.



In [742]:
def bounded_neighbours8(x, y, max_x, max_y, grid):
    return tuple(
        (x1, y1) for x1, y1 in neighbours8(x, y)
        if x1 >= 0 and x1 < max_x and y1 >= 0 and y1 < max_y
    )


OCCUPIED = '#'
EMPTY = 'L'
FLOOR = '.'


def advance_time(seat_layout, find_neighbours, occupied_limit):
    # clone layout to a new list
    next_layout = [row[:] for row in seat_layout]
    max_x = len(seat_layout[0])
    max_y = len(seat_layout)

    coordinates = product(range(len(seat_layout[0])), range(len(seat_layout)))

    for x, y in coordinates:
        if seat_layout[y][x] == FLOOR:
            continue

        neighbours = [
            seat_layout[y1][x1]
            for x1, y1 in find_neighbours(x, y, max_x, max_y, seat_layout)
        ]
        occupied = quantify(neighbours, lambda x: x == OCCUPIED)
        
        if seat_layout[y][x] == EMPTY and occupied == 0:
            next_layout[y][x] = OCCUPIED
        elif seat_layout[y][x] == OCCUPIED and occupied >= occupied_limit:
            next_layout[y][x] = EMPTY
    return next_layout
   

def advance_until_stable(seat_layout, find_neighbours=bounded_neighbours8, occupied_limit=4):
    prev_layout = seat_layout
    for t in count():
        next_layout = advance_time(prev_layout, find_neighbours, occupied_limit)
        if cat(map(cat, prev_layout)) == cat(map(cat, next_layout)):
            break

        prev_layout = next_layout

    return next_layout

input11 = list(Input(11, lambda line: list(line.strip())))
quantify(
    cat(map(cat, advance_until_stable(input11))),
    lambda x: x == OCCUPIED
)

2277

In [870]:
assert _ == 2277, "Day 11.1"

In [236]:
# part 2

def bounded_neighbours8_sight(x, y, max_x, max_y, grid):
    # Build iterators of each explorable direction
    # this has to be done within a lambda otherwise the generators
    # will have the final value of dx and dy.
    build_sightline = lambda dx, dy: ((x + d * dx, y + d * dy) for d in count(1))
    sightlines = [
        build_sightline(dx, dy)
        for (dx, dy) in NEIGHBOUR8_DELTAS
    ]
    
    neighbours = []

    # Explore each sightline until we reach the end of it
    for sightline in sightlines:
        for x1, y1 in sightline:
            try:
                if grid[y1][x1] != FLOOR:
                    neighbours.append((x1, y1))
                    break
            except IndexError:
                # Sightline out of bounds
                break
    return neighbours


quantify(
    cat(map(cat, advance_until_stable(input11, bounded_neighbours8_sight, 5))),
    lambda x: x == OCCUPIED
)

2026

In [ ]:
assaert _ == 2066, 'Day 11.2'

# [Day 12: Rain Risk](https://adventofcode.com/2020/day/12)

We're tasked with following a set of actions on a cartesian grid. I choose to represent our current heading and position as imaginary numbers, making rotations extremely easy to compute. As a result, the second part was very simple! (Which was a nice surprise.)

In [749]:
def turn(degrees, pos):
    rotation = degrees // 90 
    return pos * pow(1j, rotation)


def follow_actions(actions):
    heading = 1 + 0j
    posisition = 0 + 0j
    
    action_map = {'N': 1j, 'S': -1j, 'E': 1, 'W': -1}


    for action, value in actions:
        if action in action_map:        posisition += value * action_map[action]
        elif action == 'L':             heading = turn(value, heading)
        elif action == 'R':             heading = turn(-value, heading)
        elif action == 'F':             posisition += value * heading

    return int(abs(posisition.real) + abs(posisition.imag))


parse_line = lambda line: (line[0], int(line[1:]))

actions = Input(12, parse_line)
follow_actions(actions)


445

In [947]:
assert _ == 445, 'Day 12.2'

In [746]:
# part 2
def follow_actions(actions):
    way_point = 10 + 1j
    posisition = 0 + 0j

    action_map = {'N': 1j, 'S': -1j, 'E': 1, 'W': -1}

    for action, value in actions:
        if action in action_map:    way_point += value * action_map[action]
        elif action == 'L':         way_point = turn(value, way_point)
        elif action == 'R':         way_point = turn(-value, way_point)
        elif action == 'F':         posisition += way_point * value
        else:                       raise Exception('unkown action')

    return int(abs(posisition.real) + abs(posisition.imag))

follow_actions(actions)

42495

In [998]:
assert _ == 42495, 'Day 12.2'

# [Day 13: Shuttle Search](https://adventofcode.com/2020/day/13)

This problem was of a new breed to me, and quite annoying to solve. I broke, and had to check how to solve it (it was clear that the bus ids being prime was a crucial factor, but I could not see why). The solution was to create a set of equations of the form

```
    x = a_i (mod n1),
```

(with x in this case as 0), and then use the [chinese remainder theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem) to find the solution.

In [753]:
def find_earliest_bus(earliest_departure_time, bus_ids):
    valid_bus_ids = [int(bus_id) for bus_id in bus_ids if bus_id != 'x']
    closest_bus_id = min(valid_bus_ids, key=lambda bus_id: wait_for_bus(bus_id, earliest_departure_time))
    return closest_bus_id * wait_for_bus(closest_bus_id, earliest_departure_time)


def wait_for_bus(bus_id, t):
    return 0 if t % bus_id == 0 else bus_id - t % bus_id


def parse_input(lines):
    return (int(lines[0]), lines[1].split(','))

input13 = parse_input(Input(13))
find_earliest_bus(*input13)


4782

In [14]:
assert _ == 4782, 'Day 13.1'

In [755]:
# part 2
# Code for the chinese remainder was taken from the internet, I forget where. :(
def chinese_remainder(n, a):
    sum = 0
    prod = reduce(lambda a, b: a*b, n)
    for n_i, a_i in zip(n, a):
        p = prod // n_i
        sum += a_i * mul_inv(p, n_i) * p
    return sum % prod 
 
 
def mul_inv(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1: return 1
    while a > 1:
        q = a // b
        a, b = b, a%b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0: x1 += b0
    return x1

def find_t(bus_ids):
    bus_ids = tuple(
        (int(bus_id), int(bus_id) - idx)
        for idx, bus_id in enumerate(bus_ids)
        if bus_id != 'x'
    )

    n, a = zip(*bus_ids)

    return chinese_remainder(n, a)


find_t(input13[1])

1118684865113056

In [ ]:
assert _ == 1118684865113056, 'Day 13.2'

# [Day 14: Docking Data](https://adventofcode.com/2020/day/14)

Nothing too special about this one, I think my solution to the second part is a bit heavy handed - there's probably a simpler approach to finding all the possible masks.

In [756]:
def apply_mask_v1(mask, location, value):
    ones = int(mask.replace('X', '0'), 2)
    zeros = int(mask.replace('X', '1'), 2)
    return [(location, (value | ones) & zeros)]


def run_mask(ops, apply_mask):
    memory = {}

    mask = ''
    for opname, opvalue in ops:
        if opname == 'mask':
            mask = opvalue
        else:
            loc, val = opvalue
            for location, value in apply_mask(mask, loc, val):
                memory[location] = value
    
    return sum(memory.values())


def parse_input(lines):
    ops = []
    
    for line in lines:
        if line.startswith('mask'):
            op = 'mask'
            val = line.split('=')[1].strip()
        elif line.startswith('mem'):
            op = 'mem'
            val = mapt(int, re.findall('\d+', line))
        else:
            raise Exception(f'Unknown operation, {line}')
        ops.append((op, val))

    return ops

input14 = parse_input(Input(14))
run_mask(input14, apply_mask_v1)

14862056079561

In [ ]:
assert _ == 14862056079561, 'Day 14'

In [143]:
# part 2
def expland_floating_mask(mask, masks=None):
    if masks is None:
        masks = []

    if 'X' not in mask:
        masks.append(mask)
        return masks

    for char in '01':
        new_mask = mask.replace('X', char, 1)
        expland_floating_mask(new_mask, masks)
    
    return masks


def apply_mask_v2(mask, location, value):
    ones = int(mask.replace('X', '1'), 2)
    val = location | ones

    # Now that we've applied the static ones, as we'll be AND-ing the
    # new masks, we can ignore the zeros by replacing them with 1s
    mask = mask.replace('0', '1')
    new_locations = [
        val & int(new_mask, 2)
        for new_mask in expland_floating_mask(mask)
    ]
    return zip(new_locations, [value] * len(new_locations))


run_mask(input14, apply_mask_v2)

3296185383161

In [136]:
assert _ == 3296185383161, 'Day 14.2'

# [Day 15: Rambunctious Recitation](https://adventofcode.com/2020/day/15)

Again, nothing particularly interesting in this solution, but slightly surprised that the same answer for part 1 could be used for part 2. I imagine if a less efficient approach would be used, there'd be a problem. 

In [757]:
def play_counting_game(starting_numbers, limit):
    last_spoken = {}

    for idx, num in enumerate(starting_numbers):
        last_spoken[num] = idx + 1

    for idx in range(idx + 1, limit):
        last_idx = last_spoken.get(num, idx)
        last_spoken[num] = idx
        num = idx - last_idx

    return num

play_counting_game([14,8,16,0,1,17], limit=2020)

240

In [ ]:
assert _ == 240, 'Day 15.1'

In [758]:
# part 2
play_counting_game([14,8,16,0,1,17], limit=30000000)

505

In [760]:
assert _ == 505, 'Day 15.2'

# [Day 16: Ticket Translation](https://adventofcode.com/2020/day/16)

This problem, although quite simple, took me a while thanks to abusing python's loose typing (/ my idiocy). I was checking the validity of a ticket by returning the value that did not match any rule - treating truthy values (i.e. postive ints) as a failing ticket. Unfortunately, my input had zeros in, which should have failed. But, 0 is considered falsey, so I was erroronously counting invalid tickets. Painful!

In [396]:
Rule = namedtuple('Rule', 'name,ranges')


def apply_rule(rule, val):
    return (rule.ranges[0] <= val <= rule.ranges[1] or
            rule.ranges[2] <= val <= rule.ranges[3])


def is_valid_ticket(rules, ticket) -> (bool, int):
    for val in ticket:
        for rule in rules:
            if apply_rule(rule, val):
                break
        else:
            return False, val
    return True, None


def find_error_rate(rules, tickets):
    error_rate = 0
    for ticket in tickets:
        is_valid, invalid_field = is_valid_ticket(rules, ticket)
        if not is_valid:
            error_rate += invalid_field
    return error_rate



def parse_input(lines):
    rules = []

    lines_iter = iter(lines)
    for line in lines_iter:
        if not line:
            break
        match = re.match('(.*): (\d+)-(\d+) or (\d+)-(\d+)', line)
        rule_name = match.groups()[0]
        ranges = tuple(int(n) for n in match.groups()[1:])

        rules.append(
            Rule(rule_name, ranges)
        )

    parse_ticket = lambda l: mapt(int, re.findall('\d+', l))
    parse_ticket = lambda l: mapt(int, l.split(','))
    next(lines_iter)
    our_ticket = parse_ticket(next(lines_iter))
    next(lines_iter)

    next(lines_iter)
    other_tickets = mapt(parse_ticket, lines_iter)

    return rules, our_ticket, other_tickets

input16 = parse_input(Input(16))

find_error_rate(input16[0], input16[2])

27802

In [367]:
assert _ == 27802, 'Day 16.1'

In [413]:
# part 2
def match_fields(rules, tickets):
    valid_tickets = [
        ticket
        for ticket in tickets
        if is_valid_ticket(rules, ticket)[0]
    ]

    num_fields = len(valid_tickets[0])
    fields = dict(
        (rule.name, set(range(num_fields)))
        for rule in rules
    )

    for ticket in valid_tickets:
        for idx, val in enumerate(ticket):
            for rule in rules:
                if not apply_rule(rule, val):
                    # Rule failed, so it cannot apply to this index
                    fields[rule.name] -= {idx}

    # Reduce possible indexes by removing matches we're certain about
    known_matches = [
        (name, list(indexes)[0])
        for name, indexes in fields.items()
        if len(indexes) == 1
    ]

    while len(known_matches) != len(rules):
        for name, idx in known_matches:
            for rule in rules:
                fields[rule.name] -= {idx}
            
                if len(fields[rule.name]) == 1:
                    known_matches.append(
                        (rule.name, list(fields[rule.name])[0])
                    )
    
    return known_matches
    

def find_departure_code(rules, our_ticket, tickets):
    fields = match_fields(rules, tickets)
    res = 1
    for name, idx in fields:
        if name.startswith('departure'):
            res *= our_ticket[idx]
    return res


find_departure_code(*input16)

279139880759

In [407]:
assert _ == 279139880759, 'Day 16.2'

# [Day 17: Conway Cubes](https://adventofcode.com/2020/day/17)

[Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) but in three and four dimensions. I treat the alive cells as those that are present in the current set.

In [791]:
ACTIVE = '#'
INACTIVE = '.'

Point4 = namedtuple('Point', ['x', 'y', 'z', 'w'])

# All possible deltas for dimensions up to 4
DELTAS = [set(product((-1, 0, 1), repeat=d)) - {(0,) * d} for d in range(5)]


def get_neighbours(point):
    "Find all neighbours for a given point in n dimenions."
    return [
        tuple(map(operator.add, point, delta))
        for delta in DELTAS[len(point)]
    ]


def get_neighbour_counts(points):
    """Find all cells and how many of their neighbours are alive.

    As we find all the neighbours of all the cells simultaneously,
    by counting the number of time a given cell appears, we know 
    how many neighbours it currently has in the set.

    As we only track living cells, we therefore know how many live
    neighbours each one has.
    """
    return Counter(chain.from_iterable(map(get_neighbours, points)))


def perform_cycle(points):
    "Find the next state for all points."
    return set(
        point for point, alive_neighours
        in get_neighbour_counts(points).items()
        if alive_neighours == 3 or (alive_neighours == 2 and point in points)
    )


def count_active_cubes_after_cycles(points: set) -> int:
    for _ in range(6):
        points = perform_cycle(points)
    return len(points)


def parse_input(lines, num_dimensions):
    points = set([
        (x, y,) + (0,) * (num_dimensions - 2)
        for y, line in enumerate(lines)
        for x, char in enumerate(line)
        if char == '#'
    ])
    return points


input17 = parse_input(Input(17), num_dimensions=3)
count_active_cubes_after_cycles(input17)

295

In [424]:
assert _ == 295, 'Day 17.1'

In [785]:
input17 = parse_input(Input(17), num_dimensions=4)
count_active_cubes_after_cycles(input17)

1972

In [431]:
assert _ == 1972, 'Day 17.2'

# [Day 18: Operation Order](https://adventofcode.com/2020/day/18)

I've never written a full lexer / parser combination before, and perhaps this is overkill for the problem. But in doing so I learned a few things (like the [Shunting-yard algorithm](https://en.wikipedia.org/wiki/Shunting-yard_algorithm)) but it meant that the second part of the problem was extremely simple!

In [790]:
Token = namedtuple('Token', ['name', 'value'])


def math_lexer(expression):
    "Generates a token stream."    
    stream = StringIO(expression)

    token_map = {'+': 'ADD', '*': 'MUL', '(': 'OPEN_PAREN', ')': 'CLOSE_PAREN',}

    while True:
        char = stream.read(1)
        if not char:
            break

        if not char.strip():
            continue

        if char in token_map:
            yield Token(token_map[char], char)
        else:
            # we have a digit, consume the stream until we hit a break
            val = ''
            while char.isnumeric():
                val += char
                char = stream.read(1)

            if char:
                # we've read one extra char, go back one
                # Note: we can't use relative seeking on a stringIO obj
                stream.seek(stream.tell() - 1)

            yield Token('INT', int(val))


def math_parser(tokens, precedence):
    "Convert infix to postfix notation using shunting-yard algorithm"
    output = deque([])
    op_stack = []

    for token in tokens:
        if token.name == 'INT':
            output.append(token)
        elif token.name in ('ADD', 'MUL'):
            # Move all operators from stack onto the output
            while (
                op_stack and
                op_stack[-1].name in ('ADD', 'MUL') and
                precedence[op_stack[-1].name] >= precedence[token.name]
            ):
                output.append(op_stack.pop())
            op_stack.append(token)
        elif token.name == 'OPEN_PAREN':
            op_stack.append(token)
        elif token.name == 'CLOSE_PAREN':
            # move all operators within the paren to the output
            while True:
                op_token = op_stack.pop()
                if op_token.name == 'OPEN_PAREN':
                    break
                output.append(op_token)

    # Move all remaining operations onto the output stack
    while op_stack:
        output.append(op_stack.pop())

    return output


def evaluate_expression(expression, precedence):
    tokens = math_lexer(expression)
    output_stack = math_parser(tokens, precedence)

    result_stack = []
    ops = {'ADD': operator.add, 'MUL': operator.mul}

    for token in output_stack:
        if token.name in ('ADD', 'MUL'):
            lhs = result_stack.pop()
            rhs = result_stack.pop()
            out = ops[token.name](lhs, rhs)
            result_stack.append(out)
        else:
            result_stack.append(token.value)
    return result_stack[0]


def sum_expressions(expressions, precedence):
    return sum(
         evaluate_expression(expression, precedence)
         for expression in expressions
    )


input18 = Input(18)
sum_expressions(input18, precedence={'ADD': 1, 'MUL': 1})

701339185745

In [ ]:
assert _ == 701339185745, 'Day 18.1'

In [564]:
sum_expressions(input18, precedence={'ADD': 2, 'MUL': 1})

4208490449905

In [ ]:
assert _ == 4208490449905, 'Day 18.2'

# [Day 19: Monster Messages](https://adventofcode.com/2020/day/19)

I initially parsed all of the rules and generated all the possible matches for the grammar. In part two however, modifying the grammar causes it to be infinite, so rather than attempting to generated matches to compare against, I rewrote to instead apply a rule to each message.

The nature of the recursion means that alternatives are added to the end of a composition, so a depth-first approach can be used.

By using generators, we don't need to fully explore a rule (sidestepping their infinite nature) and can simply check a single part of it before moving to the next.

Each rule falls into the following categories,

1. It is a terminator
2. It is a composition of one or more sets of rules 

as we want to avoid materializing the full grammar set we can check letter by letter, removing the leading character on a successful match or stopping the generation as we cannot progress.


In [732]:
def generate_matches(rule_id, grammar, string):
    if isinstance(grammar[rule_id], list):
        # investigate the composition of sub rules
        for rule_ids in grammar[rule_id]:
            yield from check_rules(rule_ids, grammar, string)
    else:
        if string and string[0] == grammar[rule_id]:
            # terminator matches, reduce string
            yield string[1:]    
    # Nothing matches, terminate.


def check_rules(rule_ids, grammar, string):
    if not rule_ids:
        # no rules to check, we cannot go any further
        yield string
    else:
        # check the left most rule, and continue
        rule_id, *rule_ids = rule_ids
        # for each sub rule, follow 
        for string in generate_matches(rule_id, grammar, string):
            yield from check_rules(rule_ids, grammar, string)


def is_member(grammar, string):
    # membership is determined if the rules consume the entire string
    return any(m == '' for m in generate_matches('0', grammar, string))


def parse_input(lines):
    rules = {}

    for idx, line in enumerate(lines):
        if not line:
            break

        rule_id, rule = line.split(': ')

        if rule.startswith('"'):
            # terminator rule
            rules[rule_id] = rule[1:-1]
        else:
            rules[rule_id] = [
                comp.split(' ') for comp in rule.split(' | ')
            ]
    
    data = lines[idx + 1:]
    return rules, data
    


rules, data = parse_input(Input(19))
sum([is_member(rules, d) for d in data])

111

In [722]:
assert _ == 111, 'Day 19.1'

In [733]:
# part 2
rules = {**rules, '8': [['42'], ['42', '8']], '11': [['42', '31'], ['42', '11', '31']]}
sum([is_member(rules, d) for d in data])

343

In [ ]:
assert _ == 343, 'Day 19.2'